In [1]:
#%cd ~ && rm -rf NAS_LLM
!git clone -b features https://github.com/WpythonW/NAS_LLM.git
!uv pip install --system "numpy<2" optuna
%cd NAS_LLM/Informer2020

Cloning into 'NAS_LLM'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 280 (delta 38), reused 70 (delta 18), pack-reused 175 (from 1)
Receiving objects: 100% (280/280), 3.31 MiB | 7.93 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Using Python 3.12.12 environment at: /usr
Resolved 12 packages in 204ms
Prepared 3 packages in 344ms
Uninstalled 1 package in 29ms
Installed 3 packages in 19ms
 + colorlog==6.10.1
 - numpy==2.0.2
 + numpy==1.26.4
 + optuna==4.6.0
/content/NAS_LLM/Informer2020


In [1]:
%cd NAS_LLM/Informer2020
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
import os, optuna
import torch
import numpy as np
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
import sys
import json
from pathlib import Path
from datetime import datetime

/content/NAS_LLM/Informer2020


In [2]:
!rm "/content/NAS_LLM/Informer2020/ETTh2;pred_len=48;OPTUNA.json"
!rm "/content/NAS_LLM/Informer2020/optunaETTh2.log"

rm: cannot remove '/content/NAS_LLM/Informer2020/ETTh2;pred_len=48;OPTUNA.json': No such file or directory
rm: cannot remove '/content/NAS_LLM/Informer2020/optunaETTh2.log': No such file or directory


In [3]:
class TrialLogger:
    def __init__(self, filepath='trials.json'):
        self.filepath = Path(filepath)
        self.trials = self._load()

    def _load(self):
        return json.loads(self.filepath.read_text()) if self.filepath.exists() else []

    def _save(self):
        self.filepath.write_text(json.dumps(self.trials, indent=2))

    def add(self, trial_number, params, measures):
        self.trials.append({
            'trial': trial_number,
            'timestamp': datetime.now().isoformat(),
            **{k: float(v) for k, v in measures.items()},
            'seq_len': params['seq_len'],
            'label_len': params['label_len'],
            'e_layers': params['e_layers'],
            'n_heads': params['n_heads'],
            'factor': params['factor']
        })
        self._save()

def objective(trial):
    args = dotdict()

    # Фиксированные параметры
    args.model = 'informer'
    args.data = 'custom'
    args.root_path = './'
    args.data_path = f'../data/{dataset_name}_prepared.csv'
    args.features = 'MS'
    args.target = 'OT'
    args.freq = 'h'
    args.checkpoints = './informer_checkpoints'
    args.d_model = 512
    args.d_layers = 2
    args.d_ff = 2048
    args.dropout = 0.05
    args.attn = 'prob'
    args.embed = 'timeF'
    args.activation = 'gelu'
    args.distil = True
    args.output_attention = False
    args.mix = True
    args.padding = 0
    args.batch_size = 32
    args.learning_rate = 0.0001
    args.loss = 'mse'
    args.lradj = 'type1'
    args.train_epochs = 6
    args.patience = 3
    args.num_workers = 0
    args.itr = 1
    args.des = 'optuna'
    args.use_amp = False

    # Используем обе GPU
    args.use_gpu = True
    args.use_multi_gpu = True
    args.devices = '0'
    args.device_ids = [0]
    args.gpu = 0

    # Перебираемые параметры с constraint
    args.seq_len = trial.suggest_categorical('seq_len', [96, 168, 336, 480, 720])
    args.label_len = trial.suggest_categorical('label_len', [24, 48, 96])

    # label_len должен быть <= seq_len
    if args.label_len > args.seq_len:
        raise optuna.TrialPruned()

    args.pred_len = pred_len ##########################CHANGE
    args.e_layers = trial.suggest_categorical('e_layers', [2, 3, 4, 6])
    args.n_heads = trial.suggest_categorical('n_heads', [8, 16])
    args.factor = trial.suggest_categorical('factor', [3, 5, 8, 10])

    args.enc_in = args.dec_in = 7
    args.c_out = 1
    args.detail_freq = args.freq
    args.freq = args.freq[-1:]

    exp = Exp_Informer(args)
    setting = f'informer_trial{trial.number}'

    exp.train(setting)
    mae_val, mse_val = exp.test(setting, flag='val')
    mae_test, mse_test = exp.test(setting, flag='test')

    measures = {'mae_val': mae_val, 'mse_val': mse_val, 'mae_test': mae_test, 'mse_test': mse_test}

    logger.add(trial.number, trial.params, measures)

    return mse_val

pred_len=48
dataset_name = 'ETTh2'
logger = TrialLogger(f'{dataset_name};pred_len={pred_len};OPTUNA.json')
storage = JournalStorage(JournalFileBackend('./optunaETTh2.log'))

study = optuna.create_study(direction='minimize',
                            storage=storage,
                            study_name=f'opt,pred_len={pred_len}',
                            load_if_exists=True)
study.optimize(objective, n_trials=35)

print(f'Best MSE: {study.best_value:.4f}')
print(f'Best params: {study.best_params}')

[I 2025-11-17 16:59:11,302] A new study created in Journal with name: opt,pred_len=48


Use GPU: cuda:0
train 11979
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.2410650
	speed: 0.2048s/iter; left time: 439.2133s
	iters: 200, epoch: 1 | loss: 0.4220912
	speed: 0.1407s/iter; left time: 287.6561s
	iters: 300, epoch: 1 | loss: 0.1594562
	speed: 0.1432s/iter; left time: 278.5062s
Epoch: 1 cost time: 54.82543897628784
Epoch: 1, Steps: 374 | Train Loss: 0.2419095 Vali Loss: 0.4140745 Test Loss: 0.3109758
Validation loss decreased (inf --> 0.414074).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1165842
	speed: 0.3508s/iter; left time: 621.3539s
	iters: 200, epoch: 2 | loss: 0.0888303
	speed: 0.1541s/iter; left time: 257.4599s
	iters: 300, epoch: 2 | loss: 0.1006968
	speed: 0.1550s/iter; left time: 243.5445s
Epoch: 2 cost time: 57.24120235443115
Epoch: 2, Steps: 374 | Train Loss: 0.1216049 Vali Loss: 0.3713636 Test Loss: 0.2862931
Validation loss decreased (0.414074 --> 0.371364).  Saving model ...
Updating learning rate to 5e-05
	iters:

[I 2025-11-17 17:06:01,340] Trial 0 finished with value: 0.24823059141635895 and parameters: {'seq_len': 168, 'label_len': 48, 'e_layers': 2, 'n_heads': 8, 'factor': 5}. Best is trial 0 with value: 0.24823059141635895.


test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.2838560938835144, mae:0.4222019612789154
Use GPU: cuda:0
train 11427
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.2488020
	speed: 0.4887s/iter; left time: 998.4420s
	iters: 200, epoch: 1 | loss: 0.2537175
	speed: 0.4915s/iter; left time: 955.0367s
	iters: 300, epoch: 1 | loss: 0.2245361
	speed: 0.4923s/iter; left time: 907.3310s
Epoch: 1 cost time: 175.5958697795868
Epoch: 1, Steps: 357 | Train Loss: 0.2493549 Vali Loss: 1.2145889 Test Loss: 0.6425332
Validation loss decreased (inf --> 1.214589).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1475504
	speed: 1.1145s/iter; left time: 1879.1192s
	iters: 200, epoch: 2 | loss: 0.1940771
	speed: 0.4905s/iter; left time: 777.9741s
	iters: 300, epoch: 2 | loss: 0.1457360
	speed: 0.4915s/iter; left time: 730.3312s
Epoch: 2 cost time: 175.25478100776672
Epoch: 2, Steps: 357 | Train Loss: 0.1621105 Vali Loss: 0.49

[I 2025-11-17 17:27:35,950] Trial 1 finished with value: 0.3291991353034973 and parameters: {'seq_len': 720, 'label_len': 96, 'e_layers': 2, 'n_heads': 8, 'factor': 8}. Best is trial 0 with value: 0.24823059141635895.


...

test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.3692246079444885, mae:0.4706747829914093
Use GPU: cuda:0
train 11811
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.2149648
	speed: 0.2413s/iter; left time: 510.4315s
	iters: 200, epoch: 1 | loss: 0.2129465
	speed: 0.2429s/iter; left time: 489.4192s
	iters: 300, epoch: 1 | loss: 0.1983773
	speed: 0.2428s/iter; left time: 465.0435s
Epoch: 1 cost time: 89.60728168487549
Epoch: 1, Steps: 369 | Train Loss: 0.2396620 Vali Loss: 0.4431668 Test Loss: 0.2927333
Validation loss decreased (inf --> 0.443167).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1554623
	speed: 0.5730s/iter; left time: 1000.3758s
	iters: 200, epoch: 2 | loss: 0.1150918
	speed: 0.2431s/iter; left time: 400.1189s
	iters: 300, epoch: 2 | loss: 0.0921916
	speed: 0.2431s/iter; left time: 375.8033s
Epoch: 2 cost time: 89.62586617469788
Epoch: 2, Steps: 369 | Train Loss: 0.1334009 Vali Loss: 0.287

[I 2025-11-17 22:39:45,988] Trial 33 finished with value: 0.222422257065773 and parameters: {'seq_len': 336, 'label_len': 24, 'e_layers': 2, 'n_heads': 16, 'factor': 8}. Best is trial 18 with value: 0.19755373895168304.


test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.3440275192260742, mae:0.46436649560928345
Use GPU: cuda:0
train 11811
val 1695
test 3437
	iters: 100, epoch: 1 | loss: 0.2859748
	speed: 0.3493s/iter; left time: 738.6748s
	iters: 200, epoch: 1 | loss: 0.2279546
	speed: 0.3511s/iter; left time: 707.4613s
	iters: 300, epoch: 1 | loss: 0.1594422
	speed: 0.3510s/iter; left time: 672.2557s
Epoch: 1 cost time: 129.55862069129944
Epoch: 1, Steps: 369 | Train Loss: 0.2722811 Vali Loss: 0.9367257 Test Loss: 0.4546528
Validation loss decreased (inf --> 0.936726).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2128781
	speed: 0.8258s/iter; left time: 1441.8755s
	iters: 200, epoch: 2 | loss: 0.1471602
	speed: 0.3516s/iter; left time: 578.7333s
	iters: 300, epoch: 2 | loss: 0.1476683
	speed: 0.3521s/iter; left time: 544.2791s
Epoch: 2 cost time: 129.72597002983093
Epoch: 2, Steps: 369 | Train Loss: 0.1649820 Vali Loss: 0.

[I 2025-11-17 22:55:29,018] Trial 34 finished with value: 0.49241554737091064 and parameters: {'seq_len': 336, 'label_len': 96, 'e_layers': 6, 'n_heads': 16, 'factor': 8}. Best is trial 18 with value: 0.19755373895168304.


test shape: (107, 32, 48, 1) (107, 32, 48, 1)
test shape: (3424, 48, 1) (3424, 48, 1)
mse:0.38659489154815674, mae:0.5076962113380432
Best MSE: 0.1976
Best params: {'seq_len': 336, 'label_len': 24, 'e_layers': 2, 'n_heads': 16, 'factor': 5}


In [ ]:
!rm "/kaggle/working/NAS_LLM/Informer2020/ETTh2;pred_len=168;OPTUNA.json"

In [4]:
!ls

 data				  informer_checkpoints	 optunaETTh2.log
'ETTh2;pred_len=48;OPTUNA.json'   main_informer.py	 results
 exp				  models		 utils
